# Tabla de Símbolos y Chequeo de Tipos
Notas de clase sobre Teoría de la Compilación

**Juan David Velásquez Henao**   
jdvelasq@unal.edu.co  
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

[Licencia](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/readme.md)

**Software utilizado**.

> Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org), en el cual se presenta una introducción al diseño de lectores, generadores, traductores, interpretes y compiladores. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Windows, Linux y OS X.

> Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

> Haga clic [aquí](http://nbviewer.jupyter.org/github/jdvelasq/teoria-de-la-compilacion/blob/master/01-introduccion.ipynb) para ver la última versión de este documento en nbviewer.

> Descargue la última versión de este documento, los archivos de datos y los programas en Python a su disco duro; luego, carguelos y ejecutelos en línea en [Try Jupyter!](https://try.jupyter.org)

#### Contenido

> * [Introducción](#Introducción)
    * [Objetivos](#Objetivos)
    * [Tipos de verificaciones](#Tipos-de-verificaciones)
    * [Sistemas de Tipo](#Sistemas-de-Tipo)
* [Tabla de Símbolos](#Tabla-de-Símbolos)
    * [Ejemplo 1](#Ejemplo-1)
    * [Ejemplo 2](#Ejemplo-2)
    * [Ejemplo 3](#Ejemplo-3)
    * [Ejemplo 4](#Ejemplo-4)
* [Implementación del Generador de la  Tabla de Símbolos](#Implementación-del-Generador-de-la--Tabla-de-Símbolos)
* [Verificación de Tipos](#Verificación-de-Tipos)
    * [Implementación](#Implementación)

# Introducción

[Contenido](#Contenido)

## Objetivos

[Contenido](#Contenido)

<big>**Análisis Léxico**</big>  
Reconoce tokens válidos en el programa.

<big>**Análisis Sintáctico**</big>  
Determina si el programa está escrito correctamente.

<big>**Análisis Semántico**</big>  
Última capa de revisión. Verifica propiedades del programa.

* Verificación estática vs dinámica.

* Verificación estática:
   * Flujo.
   * Unicidad.
   * Tipos.

## Tipos de verificaciones

[Contenido](#Contenido)

<big>**Verificación del Flujo.**</big>  

* Las variables debe ser declaradas antes de ser usadas.  

* La declaración de una variable debe estar acorde con su ámbito.  

* Cada trayectoria de salida retorna un valor del tipo correcto.  

<big>**Unicidad**</big>  

* Un identificador no puede ser usado en dos declaraciones diferentes en el mismo ámbito.

<big>**Verificación de tipos**</big>  

* El número de argumentos y sus tipos concuerda con el número de parámetros y tipos en la llamada a funciones.  

* Las funciones deben retornar valores del tipo apropiado.  

* Los operadores operan sobre argumentos del tipo adecuado.  

* El lado derecho de las asignaciones concuerda con el tipo del identificador.  

* Los argumentos de las declaraciones son del tipo apropiado.  

<big>**Verificación dinámica**</big>  

* Límites de tamaño de vectores y punteros.

## Sistemas de Tipo

[Contenido](#Contenido)

**Lenguajes fuertemente tipados:** 

Los tipos  de algunos elementos del código fuente se definen mediante declaración.

    int x, y;
    double z;

Los tipos de los demás elementos del código fuente se definen por inferencia.

    x * y		int * int -> int
    x + z		int + double -> double
    x < y		int < int -> bool

# Tabla de Símbolos

[Contenido](#Contenido)

Estructura de datos que recopila información sobre identificadores en un programa:

*	Nombre 

*	Clase: variable vs funcion

*	Si es función:
    *	Nombre y tipo de cada uno de sus argumentos
    *	Punto de entrada
    *	Tipo devuelto
    *	Visibilidad (ámbito)
    *	Etc.

*	Si es identificador:
    *	Visibilidad (Ámbito)
    *	Tipo.

**Recopilación de la Información.**  

* La construcción de la tabla de símbolos (TS) se realiza mediante el recorrido del árbol sintáctico.

* Cada vez que se entra en una función se hace scope += 1. El programa principal tiene scope = 0.

* Cada vez que se encuentra una declaración de función (Nodo FUNCTION) se agrega la función a la TS, se agrega su tipo y se verifica que no exista.

* Cada vez que se encuentra una declaración de variables (Nodo VAR o argumentos de una función), se agregan las variables y sus tipos a la tabla de símbolos, verificando que no hayan sido instalados previamente.

* Cada vez que se encuentra un uso de variable o llamado a función (nodo FCALL) se verifica que haya sido declarada en la TS. Note que aquí no se verifican los tipos.

---

## Ejemplo 1

[Contenido](#Contenido)

In [1]:
%%writefile example.txt

function @sq:num (x:num)
   {
   return x * x;
   }
   
write '2^2 -> ': @sq( 2 );
end

Overwriting example.txt


In [2]:
%%sh
python yylex.py example.txt

+-- TOKENTABLE
    +-- FUNCTION {lexeme: function, lineno: 1}
    +-- UFID {lexeme: @sq, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ( {lexeme: (, lineno: 1}
    +-- ID {lexeme: x, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ) {lexeme: ), lineno: 1}
    +-- { {lexeme: {, lineno: 2}
    +-- RETURN {lexeme: return, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- * {lexeme: *, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- ; {lexeme: ;, lineno: 3}
    +-- } {lexeme: }, lineno: 4}
    +-- WRITE {lexeme: write, lineno: 6}
    +-- STR {lexeme: '2^2 -> ', lineno: 6}
    +-- : {lexeme: :, lineno: 6}
    +-- UFID {lexeme: @sq, lineno: 6}
    +-- ( {lexeme: (, lineno: 6}
    +-- NUM {lexeme: 2, lineno: 6}
    +-- ) {lexeme: ), lineno: 6}
    +-- ; {lexeme: ;, lineno: 6}
    +-- END {lexeme: end, lineno: 7}
    +-- END {lexeme: end, lineno: 1}


In [3]:
%%sh 
python yyparse.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    |   +-- FUNCTION {datatype: num, lineno: 1, scope: 1, value: @sq}
    |       +-- ARGS {lineno: 1}
    |       |   +-- ID {datatype: num, lineno: 1, scope: 1, value: x}
    |       +-- BLOCK {lineno: 3}
    |           +-- RETURN {datatype: num, lineno: 3, scope: 1, value: __return__}
    |               +-- * {datatype: num, lineno: 3}
    |                   +-- ID {lineno: 3, scope: 1, value: x}
    |                   +-- ID {lineno: 3, scope: 1, value: x}
    +-- MAINPROG {lineno: 6}
        +-- WRITE {lineno: 6}
        |   +-- STR {datatype: str, lineno: 6, value: '2^2 -> '}
        |   +-- UFCALL {lineno: 6, value: @sq}
        |       +-- NUM {datatype: num, lineno: 6, value: 2}
        +-- END {lineno: 7}


In [4]:
%%sh
python yysymboltab.py example.txt

+-- SYMBOLTABLE
    +-- $chs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $abs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $sgn {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- @sq {argstype: ['num'], datatype: num, entrylabel: 0, kind: ufun, nargs: 1, scope: 1}
    +-- __return__ {datatype: num, kind: uvar, location: 0, scope: 1}
    +-- x {datatype: num, kind: uvar, location: 1, scope: 1}


---

## Ejemplo 2

[Contenido](#Contenido)

In [5]:
%%writefile example.txt

var i:num;
i := 1;

while (i < 5) do
    {
    write 'i = ': i;
    i := i + 1;
    }

end

Overwriting example.txt


In [6]:
%%sh
python yylex.py example.txt

+-- TOKENTABLE
    +-- VAR {lexeme: var, lineno: 1}
    +-- ID {lexeme: i, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ; {lexeme: ;, lineno: 1}
    +-- ID {lexeme: i, lineno: 2}
    +-- := {lexeme: :=, lineno: 2}
    +-- NUM {lexeme: 1, lineno: 2}
    +-- ; {lexeme: ;, lineno: 2}
    +-- WHILE {lexeme: while, lineno: 4}
    +-- ( {lexeme: (, lineno: 4}
    +-- ID {lexeme: i, lineno: 4}
    +-- < {lexeme: <, lineno: 4}
    +-- NUM {lexeme: 5, lineno: 4}
    +-- ) {lexeme: ), lineno: 4}
    +-- DO {lexeme: do, lineno: 4}
    +-- { {lexeme: {, lineno: 5}
    +-- WRITE {lexeme: write, lineno: 6}
    +-- STR {lexeme: 'i = ', lineno: 6}
    +-- : {lexeme: :, lineno: 6}
    +-- ID {lexeme: i, lineno: 6}
    +-- ; {lexeme: ;, lineno: 6}
    +-- ID {lexeme: i, lineno: 7}
    +-- := {lexeme: :=, lineno: 7}
    +-- ID {lexeme: i, lineno: 7}
    +-- + {lexeme: +, lineno: 7}
    +-- NUM {lexeme: 1, lineno: 7}
    +-- ; {lexeme: ;, lineno: 7}
    +--

In [7]:
%%sh
python yyparse.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    +-- MAINPROG {lineno: 1}
        +-- VAR {lineno: 1}
        |   +-- ID {datatype: num, lineno: 1, scope: 0, value: i}
        +-- ASSIGN {lineno: 2, scope: 0, value: i}
        |   +-- NUM {datatype: num, lineno: 2, value: 1}
        +-- WHILE {lineno: 4}
        |   +-- < {datatype: bool, lineno: 4}
        |   |   +-- ID {lineno: 4, scope: 0, value: i}
        |   |   +-- NUM {datatype: num, lineno: 4, value: 5}
        |   +-- BLOCK {lineno: 6}
        |       +-- WRITE {lineno: 6}
        |       |   +-- STR {datatype: str, lineno: 6, value: 'i = '}
        |       |   +-- ID {lineno: 6, scope: 0, value: i}
        |       +-- ASSIGN {lineno: 7, scope: 0, value: i}
        |           +-- + {datatype: num, lineno: 7}
        |               +-- ID {lineno: 7, scope: 0, value: i}
        |               +-- NUM {datatype: num, lineno: 7, value: 1}
        +-- END {lineno: 10}


In [8]:
%%sh
python yysymboltab.py example.txt

+-- SYMBOLTABLE
    +-- $chs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $abs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $sgn {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- i {datatype: num, kind: uvar, location: 0, scope: 0}


## Ejemplo 3

[Contenido](#Contenido)

In [9]:
%%writefile example.txt

var n:num, i:num, j:num;

n := 5;
i := 1;

while (true) do
  {
  j := 1;
  while (true) do
    {
    write 'i = ': i;
    write 'j = ': j;
    j := j + 1;
    }
  i := i + 1;
  }

end

Overwriting example.txt


In [10]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- VAR {lexeme: var, lineno: 1}
    +-- ID {lexeme: n, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- , {lexeme: ,, lineno: 1}
    +-- ID {lexeme: i, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- , {lexeme: ,, lineno: 1}
    +-- ID {lexeme: j, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ; {lexeme: ;, lineno: 1}
    +-- ID {lexeme: n, lineno: 3}
    +-- := {lexeme: :=, lineno: 3}
    +-- NUM {lexeme: 5, lineno: 3}
    +-- ; {lexeme: ;, lineno: 3}
    +-- ID {lexeme: i, lineno: 4}
    +-- := {lexeme: :=, lineno: 4}
    +-- NUM {lexeme: 1, lineno: 4}
    +-- ; {lexeme: ;, lineno: 4}
    +-- WHILE {lexeme: while, lineno: 6}
    +-- ( {lexeme: (, lineno: 6}
    +-- BOOL {lexeme: true, lineno: 6}
    +-- ) {lexeme: ), lineno: 6}
    +-- DO {lexeme: do, lineno: 6}
    +-- { {lexeme: {, lineno: 7}
    +-- ID {lexeme: j, lineno: 8}


In [11]:
%%sh
python yyparse.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    +-- MAINPROG {lineno: 1}
        +-- VAR {lineno: 1}
        |   +-- ID {datatype: num, lineno: 1, scope: 0, value: n}
        |   +-- ID {datatype: num, lineno: 1, scope: 0, value: i}
        |   +-- ID {datatype: num, lineno: 1, scope: 0, value: j}
        +-- ASSIGN {lineno: 3, scope: 0, value: n}
        |   +-- NUM {datatype: num, lineno: 3, value: 5}
        +-- ASSIGN {lineno: 4, scope: 0, value: i}
        |   +-- NUM {datatype: num, lineno: 4, value: 1}
        +-- WHILE {lineno: 6}
        |   +-- BOOL {datatype: bool, lineno: 6, value: True}
        |   +-- BLOCK {lineno: 8}
        |       +-- ASSIGN {lineno: 8, scope: 0, value: j}
        |       |   +-- NUM {datatype: num, lineno: 8, value: 1}
        |       +-- WHILE {lineno: 9}
        |       |   +-- BOOL {datatype: bool, lineno: 9, value: True}
        |       |   +-- BLOCK {lineno: 11}
        |       |       +-- WRITE {lineno: 11}
        |       |       |   +-- S

In [12]:
%%sh
python yysymboltab.py example.txt

+-- SYMBOLTABLE
    +-- $chs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $abs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $sgn {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- n {datatype: num, kind: uvar, location: 0, scope: 0}
    +-- i {datatype: num, kind: uvar, location: 1, scope: 0}
    +-- j {datatype: num, kind: uvar, location: 2, scope: 0}


## Ejemplo 4

[Contenido](#Contenido)

In [29]:
%%writefile example.txt

function @min:num (x:num, y:num)
   {
   when (x < y) do return x;
   return y;
   }

function @max:num (x:num, y:num)   
   {
   if (x < y) do
       {
       return x;
       }
   else
      {                 
      return y;
      }
   }

write 'min 1 -> ': @min(1,2);
write 'max 2 -> ': @max(1,2);

end

Overwriting example.txt


In [30]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- FUNCTION {lexeme: function, lineno: 1}
    +-- UFID {lexeme: @min, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ( {lexeme: (, lineno: 1}
    +-- ID {lexeme: x, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- , {lexeme: ,, lineno: 1}
    +-- ID {lexeme: y, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ) {lexeme: ), lineno: 1}
    +-- { {lexeme: {, lineno: 2}
    +-- WHEN {lexeme: when, lineno: 3}
    +-- ( {lexeme: (, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- < {lexeme: <, lineno: 3}
    +-- ID {lexeme: y, lineno: 3}
    +-- ) {lexeme: ), lineno: 3}
    +-- DO {lexeme: do, lineno: 3}
    +-- RETURN {lexeme: return, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- ; {lexeme: ;, lineno: 3}
    +-- RETURN {lexeme: return, lineno: 4}
    +-- ID {lexeme: y, lineno: 4}
    +-- ; {lexeme: ;, lineno: 4}
    +-- } {le

In [31]:
%%sh
python yyparse.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    |   +-- FUNCTION {datatype: num, lineno: 1, scope: 1, value: @min}
    |   |   +-- ARGS {lineno: 1}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: x}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: y}
    |   |   +-- BLOCK {lineno: 3}
    |   |       +-- WHEN {lineno: 3}
    |   |       |   +-- < {datatype: bool, lineno: 3}
    |   |       |   |   +-- ID {lineno: 3, scope: 1, value: x}
    |   |       |   |   +-- ID {lineno: 3, scope: 1, value: y}
    |   |       |   +-- RETURN {datatype: num, lineno: 3, scope: 1, value: __return__}
    |   |       |       +-- ID {lineno: 3, scope: 1, value: x}
    |   |       +-- RETURN {datatype: num, lineno: 4, scope: 1, value: __return__}
    |   |           +-- ID {lineno: 4, scope: 1, value: y}
    |   +-- FUNCTION {datatype: num, lineno: 7, scope: 2, value: @max}
    |       +-- ARGS {lineno: 7}
    |       |   +-- ID {datatype: num, lineno: 7, scope: 2, v

In [32]:
%%sh
python yysymboltab.py example.txt

+-- SYMBOLTABLE
    +-- $chs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $abs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $sgn {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- @min {argstype: ['num', 'num'], datatype: num, entrylabel: 0, kind: ufun, nargs: 2, scope: 1}
    +-- __return__ {datatype: num, kind: uvar, location: 0, scope: 1}
    +-- x {datatype: num, kind: uvar, location: 1, scope: 1}
    +-- y {datatype: num, kind: uvar, location: 2, scope: 1}
    +-- @max {argstype: ['num', 'num'], datatype: num, entrylabel: 1, kind: ufun, nargs: 2, scope: 2}
    +-- __return__ {datatype: num, kind: uvar, location: 3, scope: 2}
    +-- x {datatype: num, kind: uvar, location: 4, scope: 2}
    +-- y {datatype: num, kind: uvar, location: 5, scope: 2}


---

# Implementación del Generador de la  Tabla de Símbolos

[Contenido](#Contenido)

In [35]:
%%sh
pygmentize -O linenos=1 -g yysymboltab.py

0001: ###< 2016-08-28 17:04:40.183908 >###
0002: 
0003: #
0004: #  yysimboltab.py
0005: #    generador de la tabla de simbolos
0006: #
0007: 
0008: import shlex, sys, pickle
0009: import dataTree as dt
0010: 
0011: 
0012: # generacion de la tabla de simbolos
0013: def yysymboltab(filename, quiet = False):
0014: 
0015:     # lee la estructura de datos
0016:     with open(filename + '.dataTree', 'rb') as f:
0017:         DATA = pickle.load(f)
0018: 
0019:     SOURCECODE  = DATA.find('SOURCECODE')
0020:     TOKENTABLE  = DATA.find('TOKENTABLE')
0021:     SYNTAXTREE  = DATA.find('SYNTAXTREE')
0022:     SYMBOLTABLE = DATA.find('SYMBOLTABLE')
0023: 
0024:     def yyerror(node, msg):
0025:         # el simbolo ya existe en la tabla de simbolos
0026:         lineno = node.attrib['lineno']
0027:         symbol = node.attrib['value']
0028:         txt = '{}:Semantic error at line {}: '.format(filename, lineno)
0029:         txt += 'Symbol <{}> {}'.format(symbol, msg)
0030:         # msg += '--> 

---

# Verificación de Tipos

[Contenido](#Contenido)

* Se hace recorriendo el árbol sintáctico.

* Se asignan tipos a algunos nodos del árbol sintáctico durante su construcción.

* Se construye la TS recopilando información sobre los tipos.

* Se recorre el árbol agregando los tipos a los nodos y verificando operaciones

Para facilitar la verificación de tipo se agregan propiedades a los nodos de los árboles, tal como el scope.

In [38]:
%%writefile example.txt

function @min:num (x:num, y:num)
   {
   when (x < y) do return x;
   return y;
   }

function @max:num (x:num, y:num)   
   {
   if (x < y) do
       {
       return x;
       }
   else
      {                 
      return y;
      }
   }

write 'min 1 -> ': @min(1,2);
write 'max 2 -> ': @max(1,2);

end

Overwriting example.txt


In [39]:
%%sh 
python yylex.py example.txt

+-- TOKENTABLE
    +-- FUNCTION {lexeme: function, lineno: 1}
    +-- UFID {lexeme: @min, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ( {lexeme: (, lineno: 1}
    +-- ID {lexeme: x, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- , {lexeme: ,, lineno: 1}
    +-- ID {lexeme: y, lineno: 1}
    +-- : {lexeme: :, lineno: 1}
    +-- DATATYPE {lexeme: num, lineno: 1}
    +-- ) {lexeme: ), lineno: 1}
    +-- { {lexeme: {, lineno: 2}
    +-- WHEN {lexeme: when, lineno: 3}
    +-- ( {lexeme: (, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- < {lexeme: <, lineno: 3}
    +-- ID {lexeme: y, lineno: 3}
    +-- ) {lexeme: ), lineno: 3}
    +-- DO {lexeme: do, lineno: 3}
    +-- RETURN {lexeme: return, lineno: 3}
    +-- ID {lexeme: x, lineno: 3}
    +-- ; {lexeme: ;, lineno: 3}
    +-- RETURN {lexeme: return, lineno: 4}
    +-- ID {lexeme: y, lineno: 4}
    +-- ; {lexeme: ;, lineno: 4}
    +-- } {le

In [40]:
%%sh 
python yyparse.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    |   +-- FUNCTION {datatype: num, lineno: 1, scope: 1, value: @min}
    |   |   +-- ARGS {lineno: 1}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: x}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: y}
    |   |   +-- BLOCK {lineno: 3}
    |   |       +-- WHEN {lineno: 3}
    |   |       |   +-- < {datatype: bool, lineno: 3}
    |   |       |   |   +-- ID {lineno: 3, scope: 1, value: x}
    |   |       |   |   +-- ID {lineno: 3, scope: 1, value: y}
    |   |       |   +-- RETURN {datatype: num, lineno: 3, scope: 1, value: __return__}
    |   |       |       +-- ID {lineno: 3, scope: 1, value: x}
    |   |       +-- RETURN {datatype: num, lineno: 4, scope: 1, value: __return__}
    |   |           +-- ID {lineno: 4, scope: 1, value: y}
    |   +-- FUNCTION {datatype: num, lineno: 7, scope: 2, value: @max}
    |       +-- ARGS {lineno: 7}
    |       |   +-- ID {datatype: num, lineno: 7, scope: 2, v

In [41]:
%%sh 
python yysymboltab.py example.txt

+-- SYMBOLTABLE
    +-- $chs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $abs {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- $sgn {argstype: ['num'], datatype: num, kind: bfun, nargs: 1}
    +-- @min {argstype: ['num', 'num'], datatype: num, entrylabel: 0, kind: ufun, nargs: 2, scope: 1}
    +-- __return__ {datatype: num, kind: uvar, location: 0, scope: 1}
    +-- x {datatype: num, kind: uvar, location: 1, scope: 1}
    +-- y {datatype: num, kind: uvar, location: 2, scope: 1}
    +-- @max {argstype: ['num', 'num'], datatype: num, entrylabel: 1, kind: ufun, nargs: 2, scope: 2}
    +-- __return__ {datatype: num, kind: uvar, location: 3, scope: 2}
    +-- x {datatype: num, kind: uvar, location: 4, scope: 2}
    +-- y {datatype: num, kind: uvar, location: 5, scope: 2}


In [42]:
%%sh 
python yytypecheck.py example.txt

+-- SYNTAXTREE
    +-- FUNCTIONDECL {lineno: 1}
    |   +-- FUNCTION {datatype: num, lineno: 1, scope: 1, value: @min}
    |   |   +-- ARGS {lineno: 1}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: x}
    |   |   |   +-- ID {datatype: num, lineno: 1, scope: 1, value: y}
    |   |   +-- BLOCK {lineno: 3}
    |   |       +-- WHEN {lineno: 3}
    |   |       |   +-- < {datatype: bool, lineno: 3}
    |   |       |   |   +-- ID {datatype: num, lineno: 3, scope: 1, value: x}
    |   |       |   |   +-- ID {datatype: num, lineno: 3, scope: 1, value: y}
    |   |       |   +-- RETURN {datatype: num, lineno: 3, scope: 1, value: __return__}
    |   |       |       +-- ID {datatype: num, lineno: 3, scope: 1, value: x}
    |   |       +-- RETURN {datatype: num, lineno: 4, scope: 1, value: __return__}
    |   |           +-- ID {datatype: num, lineno: 4, scope: 1, value: y}
    |   +-- FUNCTION {datatype: num, lineno: 7, scope: 2, value: @max}
    |       +-- ARGS {lineno: 7}
 

### Implementación

[Contenido](#Contenido)

In [37]:
%%sh
pygmentize -O linenos=1 -g yytypecheck.py

0001: ###< 2016-08-28 17:04:40.184297 >###
0002: 
0003: #
0004: #  yytypecheck.py
0005: #    verificador de tipos
0006: #
0007: 
0008: import shlex, sys, pickle
0009: import dataTree as dt
0010: 
0011: 
0012: 
0013: def yytypecheck(filename, quiet=False):
0014: 
0015:     # lee la estructura de datos
0016:     with open(filename + '.dataTree', 'rb') as f:
0017:         DATA = pickle.load(f)
0018: 
0019:     SOURCECODE  = DATA.find('SOURCECODE')
0020:     TOKENTABLE  = DATA.find('TOKENTABLE')
0021:     SYNTAXTREE  = DATA.find('SYNTAXTREE')
0022:     SYMBOLTABLE = DATA.find('SYMBOLTABLE')
0023: 
0024: 
0025:     def yyerror(node):
0026:         msg = '{}:Type error at line {}: '.format(filename, node.get('lineno'))
0027:         msg += 'Invalid type <{}> found'.format(node.get('datatype'))
0028:         print(msg)
0029:         sys.exit()
0030: 
0031:     def find_symbol(symbol, scope=None):
0032:         if scope is None:
0033:             # se esta buscando una funcion
0034:           

---

[Contenido](#Contenido)